In [2]:
import keras
import pandas as pd
import tensorflow as tf
import tensorflow.compat.v1 as tf1
# # set this so LinearModel.fit won't have bug
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

#### Prepare data for model

In [3]:
x_train = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
x_eval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')

In [4]:
# 将类别转为数字
x_train['sex'].replace(('male', 'female'), (0, 1), inplace=True)
x_eval['sex'].replace(('male', 'female'), (0, 1), inplace=True)

x_train['alone'].replace(('n', 'y'), (0, 1), inplace=True)
x_eval['alone'].replace(('n', 'y'), (0, 1), inplace=True)

x_train['class'].replace(('First', 'Second', 'Third'), (1, 2, 3), inplace=True)
x_eval['class'].replace(('First', 'Second', 'Third'), (1, 2, 3), inplace=True)

# 将没用的列Drop
x_train.drop(['embark_town', 'deck'], axis=1, inplace=True)
x_eval.drop(['embark_town', 'deck'], axis=1, inplace=True)

# 将target单独拿出
y_train = x_train.pop('survived')
y_eval = x_eval.pop('survived')

In [5]:
# Data setup for TensorFlow 1 with `tf.estimator`
def _input_fn():
    return tf1.data.Dataset.from_tensor_slices((dict(x_train), y_train)).batch(32)


def _eval_input_fn():
    return tf1.data.Dataset.from_tensor_slices((dict(x_eval), y_eval)).batch(32)


FEATURE_NAMES = [
    'age', 'fare', 'sex', 'n_siblings_spouses', 'parch', 'class', 'alone'
]

feature_columns = []
for fn in FEATURE_NAMES:
    feat_col = tf1.feature_column.numeric_column(fn, dtype=tf.float32)
    feature_columns.append(feat_col)

In [6]:
# Create a method to instantiate a simplistic sample optimizer to use with our various TensorFlow 1 Estimator and TensorFlow 2 Keras models

def create_sample_optimizer(tf_version):
    if tf_version == 'tf1':
        optimizer = lambda: tf.keras.optimizers.Ftrl(
            l1_regularization_strength=0.001,
            learning_rate=tf1.train.exponential_decay(
                learning_rate=0.1,
                global_step=tf1.train.get_global_step(),
                decay_steps=10000,
                decay_rate=0.9))
    elif tf_version == 'tf2':
        optimizer = tf.keras.optimizers.Ftrl(
            l1_regularization_strength=0.001,
            learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate=0.1, decay_steps=10000, decay_rate=0.9))
    return optimizer


#### Create Model

##### Train Model JOINTLY

**Method 1**

In [ ]:
optimizer = create_sample_optimizer('tf1')

combined_estimator = tf.estimator.DNNLinearCombinedEstimator(
    head=tf.estimator.BinaryClassHead(),
    # Wide settings
    linear_feature_columns=feature_columns,
    linear_optimizer=optimizer,
    # Deep settings
    dnn_feature_columns=feature_columns,
    dnn_hidden_units=[128],
    dnn_optimizer=optimizer)

In [ ]:
combined_estimator.train(input_fn=_input_fn, steps=100)

In [ ]:
combined_estimator.evaluate(input_fn=_eval_input_fn, steps=10)

**Method 2**

In [7]:
x_train = x_train.to_numpy()
y_train = y_train.to_numpy()

In [8]:
# Create LinearModel and DNN Model as in Examples 1 and 2
optimizer = create_sample_optimizer('tf2')

# wide part
linear_model = tf.keras.experimental.LinearModel()
linear_model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
linear_model.fit(x_train, y_train, epochs=10, verbose=100)

# deep part
dnn_model = tf.keras.models.Sequential(
    [tf.keras.layers.Dense(128, activation='relu'),
     tf.keras.layers.Dense(1)])
dnn_model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 627 samples
Metal device set to: Apple M1 Pro
Epoch 1/10


2022-03-02 11:17:22.082795: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-02 11:17:22.082946: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-03-02 11:17:22.087742: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-02 11:17:22.087823: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-02 11:17:22.096724: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-02 11:17:22.104870: I tensorflow/core/grappler/o

Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10


In [9]:
combined_model = tf.keras.experimental.WideDeepModel(linear_model,
                                                               dnn_model)
combined_model.compile(
    optimizer=[optimizer, optimizer], loss='mse', metrics=['accuracy'])
combined_model.fit([x_train, x_train], y_train, epochs=10)
result = combined_model.evaluate([x_eval, x_eval], y_eval)
result = {out: result[i] for i, out in enumerate(combined_model.metrics_names)}
result

Train on 627 samples
Epoch 1/10


2022-03-02 11:17:23.902725: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-02 11:17:23.912362: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-02 11:17:23.921730: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-02 11:17:23.925911: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


627/627 [==============================] - 1s 888us/sample - loss: 554.8719 - accuracy: 0.5072
Epoch 2/10
627/627 [==============================] - 0s 180us/sample - loss: 1.0064 - accuracy: 0.6938
Epoch 3/10
627/627 [==============================] - 0s 193us/sample - loss: 0.2956 - accuracy: 0.7018
Epoch 4/10
627/627 [==============================] - 0s 174us/sample - loss: 0.2443 - accuracy: 0.7416
Epoch 5/10
627/627 [==============================] - 0s 166us/sample - loss: 0.2173 - accuracy: 0.7608
Epoch 6/10
627/627 [==============================] - 0s 167us/sample - loss: 0.2305 - accuracy: 0.7560
Epoch 7/10
627/627 [==============================] - 0s 178us/sample - loss: 0.2115 - accuracy: 0.7624
Epoch 8/10
627/627 [==============================] - 0s 172us/sample - loss: 0.1873 - accuracy: 0.7767
Epoch 9/10
627/627 [==============================] - 0s 161us/sample - loss: 0.1861 - accuracy: 0.7831
Epoch 10/10
627/627 [==============================] - 0s 160us/sample - 

/Users/xiaoziqi/miniconda3/lib/python3.8/site-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
2022-03-02 11:17:25.477631: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


{'loss': 0.19826363230293448, 'accuracy': 0.7348485}